# **Pip install**

In [1]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=de37ea5a7a4f4d23fc72f2801c36db3b24ea891b6255dcc663a4de061cd1ff54
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


# **import**

In [2]:
import numpy as np
import random
import math
# importing the QISKit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,pauli_error, depolarizing_error, thermal_relaxation_error)

# **Utils_functions**

In [3]:
def hamming_distance(list1, list2):
    # Ensure both lists have the same length
    if len(list1) != len(list2):
        raise ValueError("Bit lists must have the same length")

    # Use zip to iterate over corresponding bits in both lists
    # and count the differing bits
    distance = sum(bit1 != bit2 for bit1, bit2 in zip(list1, list2))

    return distance

def bitarray2int(bit_array):
  # Convert the bit array to a string
  bit_string = ''.join(map(str, bit_array))

  # Convert the binary string to an integer
  result = int(bit_string, 2)

  return result

def relative_error_cal(approximate_value, true_value):
    try:
        error = (approximate_value - true_value) / abs(true_value)
        return round(error,3)
    except ZeroDivisionError:
        print("Error: Division by zero. True value cannot be zero.")
        return None
def bitstr2arr(bitstr):
  return [int(i) for i in bitstr]

In [4]:
def bitarray2str(bit_array):
  # Convert the bit array to a string
  return  "".join(str(i) for i in bit_array)
# "".join(str(i) for i in alice_raw_key), "".join(str(i) for i in SA),"".join(str(i) for i in SB),"".join(str(i) for i in alice_basis),"".join(str(i) for i in bob_basis)

In [5]:
def calculate_qber(selected_bitsAlice, selected_bitsBob):
    """
    Calculate the Quantum Bit Error Rate (QBER) based on selected bits from Alice and Bob.

    Parameters:
    selected_bitsAlice (list of int): The list of bits from Alice (either 0 or 1).
    selected_bitsBob (list of int): The list of bits from Bob (either 0 or 1).

    Returns:
    float: The calculated QBER, which is the ratio of differing bits to the total number of compared bits.
    """
    if len(selected_bitsAlice) != len(selected_bitsBob):
        raise ValueError("The bit lists from Alice and Bob must be of the same length.")

    # Count the number of differing bits
    differing_bits = 0
    total_bits = len(selected_bitsAlice)

    for i in range(total_bits):
        if selected_bitsAlice[i] != selected_bitsBob[i]:
            differing_bits += 1

    # Calculate QBER as the ratio of differing bits to the total number of bits
    qber = differing_bits / total_bits
    return qber

# **SIM**

In [6]:
def sim_circuits_no_noise(circuit,shots=1):
  # sim_ideal = AerSimulator(method='automatic')
  sim_ideal = AerSimulator(method='matrix_product_state')
  job = sim_ideal.run(circuit, shots=shots)
  result_ideal = job.result()
  return result_ideal

In [7]:
from qiskit_aer.noise import (NoiseModel, depolarizing_error, amplitude_damping_error, thermal_relaxation_error)
from qiskit_aer import AerSimulator

def sim_circuits_with_noise(circuit, shots=1, L=10):
    """
    Simulate a quantum circuit with noise, where noise is influenced by the distance L.

    Parameters:
    - circuit: The quantum circuit to simulate.
    - shots: Number of shots for the simulation (default is 1).
    - L: Distance factor that influences noise (default is 10).

    Returns:
    - result: The simulation result with noise.
    """

    # Noise parameters based on distance L
    base_depolarizing_prob = 0.001  # Base depolarizing probability
    base_damping_prob = 0.002  # Base amplitude damping probability

    # Depolarizing error increases with distance
    depolarizing_prob = base_depolarizing_prob * L

    # Amplitude damping error, also distance dependent
    damping_prob = base_damping_prob * L

    # Create noise channels
    depolarizing_error_channel = depolarizing_error(depolarizing_prob, 1)  # Single-qubit depolarizing error
    amplitude_damping_channel = amplitude_damping_error(damping_prob)

    # Create a noise model and add the noise channels
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(depolarizing_error_channel, ['u1', 'u2', 'u3'])  # Apply to all gate types
    noise_model.add_all_qubit_quantum_error(amplitude_damping_channel, ['measure'])  # Apply to measurements

    # Optional: Add more channels, like phase damping, if needed
    # Here you can extend the noise model with additional channels.

    # Set up the AerSimulator with noise
    sim = AerSimulator(noise_model=noise_model, method='matrix_product_state')

    # Run the simulation
    job = sim.run(circuit, shots=shots)
    result = job.result()

    return result

In [8]:
def sim_circuits(circuit,shots=1,noise=False, L=None):
  if noise:
    results=sim_circuits_with_noise(circuit,shots=shots, L=L).get_counts()
  else:
    results=sim_circuits_no_noise(circuit,shots=shots).get_counts()


  result_dict = {key: value / shots for key, value in results.items()}
  return result_dict

# **Encoded QKD Functions**

In [9]:
def SendQ(num_qubits,alice_raw_key,alice_basis,user="Alice"):
  # Creating registers with n qubits
  qr = QuantumRegister(num_qubits, name='qr')
  crB = ClassicalRegister(num_qubits, name='crB')
  quantumChannel = QuantumCircuit(qr, crB)

  for i in range(num_qubits):
    if alice_raw_key[i] == 1 or alice_raw_key[i] == '1':
        quantumChannel.x(qr[i])
    if alice_basis[i] == 1 or alice_basis[i] == '1':
        quantumChannel.h(qr[i])

  quantumChannel.barrier(qr,label="AliceSending")
  return qr,crB,quantumChannel

def ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannel,user="Bob"):
  for i in range(num_qubits):
    if bob_basis[i] == 1 or bob_basis[i] == '1':
      quantumChannel.h(qr[i])

  quantumChannel.barrier(qr,label=f"{user}Basis")
  for i in range(num_qubits):
    quantumChannel.measure(qr[i],crB[i])
  quantumChannel.barrier(qr,label=f"{user}Measures")
  return quantumChannel

In [10]:
def Sift(num_qubits,Key,BA,BB):
  SiftedKey=""
  for i in range(num_qubits):
    if BA[i] == BB[i]:
      SiftedKey+=str(Key[i])
  return SiftedKey

In [11]:
def Sample(Key,BA,BB):
  # if(BA!=BB):
  #   raise ValueError("The bit lists from Alice and Bob must be of the same length.")

  Sampledindex=[]
  SampledKey=""
  GenKey=""
  for i in range(len(Key)):
    if BA[i] == BB[i] == '1' or BA[i] == BB[i] == 1:
      SampledKey+=str(Key[i])
      Sampledindex.append(i)
    else:
      GenKey+=str(Key[i])

  return GenKey,SampledKey,Sampledindex

In [12]:
# def __init__(self,num_qubits,shots=1):
#   num_qubits=num_qubits
#   shots=shots
#   alice_raw_key   = list(np.random.randint(2, size=num_qubits))
#   alice_basis = list(np.random.randint(2, size=num_qubits))
#   bob_basis   = list(np.random.randint(2, size=num_qubits))

# def Alice_Quantum(num_qubits):
#   alice_raw_key   = list(np.random.randint(2, size=num_qubits))
#   alice_basis = list(np.random.randint(2, size=num_qubits))
#   qr,crB,quantumChannelA2B=SendQ(num_qubits,alice_raw_key,alice_basis,user="Alice")
#   return qr,crB,quantumChannelA2B

# def Alice_Sift(self,SB):
#   #Sift
#   KsiftedA=Sift(num_qubits,alice_raw_key,alice_basis,bob_basis)
#   SA,KA=Sample(KsiftedA,alice_basis,bob_basis)
#   CheckEveA=calculate_qber(SA,SB)

#   return qr,crB,quantumChannelA2B,alice_basis,SA

# def Bob(self,SA):
#   bob_basis   = list(np.random.randint(2, size=num_qubits))
#   quantumChannelB=ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannelA2B,user="Bob")
#   resultB=list(sim_circuits(quantumChannelB,shots=1).keys())[0][::-1]
#   #Sift
#   KsiftedB=Sift(num_qubits,resultB,alice_basis,bob_basis)
#   SB,KB=Sample(KsiftedB,alice_basis,bob_basis)
#   CheckEveB=calculate_qber(SA,SB)

#   return bob_basis, SB

In [13]:
# num_qubits=100
# qr,crB,quantumChannelA2B,alice_basis,SA = Alice_QKD(num_qubits,bob_basis,SB)
# bob_basis, SB = Bob_QKD(num_qubits,alice_basis,SA)

In [14]:
def QKD_Session(num_qubits,shots=1):
  alice_raw_key   = list(np.random.randint(2, size=num_qubits))
  alice_basis = list(np.random.randint(2, size=num_qubits))
  bob_basis   = list(np.random.randint(2, size=num_qubits))



  #Quantum Channel
  qr,crB,quantumChannelA2B=SendQ(num_qubits,alice_raw_key,alice_basis,user="Alice")
  quantumChannelB=ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannelA2B,user="Bob")
  resultB=list(sim_circuits(quantumChannelB,shots=shots).keys())[0][::-1]

  #Sift
  KsiftedB=Sift(num_qubits,resultB,alice_basis,bob_basis)
  KsiftedA=Sift(num_qubits,"".join(str(i) for i in alice_raw_key),alice_basis,bob_basis)

  #ParameterEstimation
  ##Sample

  KA,SAv,SAi=Sample(KsiftedA,alice_basis,bob_basis)
  KB,SBv,SBi=Sample(KsiftedB,alice_basis,bob_basis)

  SA=(SAv,SAi)
  SB=(SBv,SBi)
  # sampleratio=0.1*num_qubits

  # SA=KsiftedA[:int(sampleratio)]
  # SB=KsiftedB[:int(sampleratio)]

  # KA=KsiftedA[int(sampleratio):]
  # KB=KsiftedB[int(sampleratio):]

  ##CheckEve
  CheckEveA=calculate_qber(SA[0],SB[0])
  CheckEveB=calculate_qber(SA[0],SB[0])
  # print(f"(CheckEveA={CheckEveA}; CheckEveB={CheckEveB})")


  return alice_raw_key, SA,SB,alice_basis,bob_basis

# "".join(str(i) for i in alice_raw_key), "".join(str(i) for i in SA),"".join(str(i) for i in SB),"".join(str(i) for i in alice_basis),"".join(str(i) for i in bob_basis)

# **Entanglment Functions**

In [15]:
def measure_cir(circuit,qr,cr,bit):
  circuit.measure(qr[bit],cr[bit])
  return circuit

In [16]:
# def aliceMeasurementChoices(num_qubits):
#   return [random.choice([0, 1, 2]) for i in range(num_qubits)] # string b of Alice

# def bobMeasurementChoices(num_qubits):
#   return [random.choice([0, 2, 3]) for i in range(num_qubits)] # string b' of Bob

def aliceMeasurementChoices(num_qubits):
  return [random.choice([1, 2, 3]) for i in range(num_qubits)] # string b of Alice

def bobMeasurementChoices(num_qubits):
  return [random.choice([1, 2, 3]) for i in range(num_qubits)] # string b' of Bob

In [17]:
def base_B2_A3(basisB3,qr,bit):
  # basis the spin projection of Alice's qubit onto the a_3 direction (standard Z basis)
  return basisB3


def base_A1(basisB1,qr,bit):
  # basis the spin projection of Alice's qubit onto the a_1 direction (X basis)
  basisB1.h(qr[bit])
  return basisB1


def base_A2_B1(basisB2,qr,bit):
  # basis the spin projection of Alice's qubit onto the a_2 direction (W basis)
  basisB2.s(qr[bit])
  basisB2.h(qr[bit])
  basisB2.t(qr[bit])
  basisB2.h(qr[bit])
  return basisB2


def base_B3(basisB4,qr,bit):
  # basis the spin projection of Bob's qubit onto the b_3 direction (V basis)
  basisB4.s(qr[bit])
  basisB4.h(qr[bit])
  basisB4.tdg(qr[bit])
  basisB4.h(qr[bit])
  return basisB4


entanglment_basis_mapA={1:base_A1, 2:base_A2_B1, 3:base_B2_A3}
entanglment_basis_mapB={1:base_A2_B1, 2:base_B2_A3, 3:base_B3}


def apply_basis(num_qubits, qc,qrA,qrB,alice_basis,bob_basis):
  for i in range(num_qubits):
    qc=entanglment_basis_mapA[alice_basis[i]](qc,qrA,i)

  for i in range(num_qubits):
    qc=entanglment_basis_mapB[bob_basis[i]](qc,qrB,i)

  qc.barrier(label="Basis")
  return qc

def apply_measurement(num_qubits,qc,qrA,qrB,crA,crB):

  for i in range(num_qubits):
    qc=measure_cir(qc,qrA,crA,i)
    qc=measure_cir(qc,qrB,crB,i)

  _=qc.barrier(label="Measure")
  return qc

def get_result(qc):
  res=sim_circuits(qc,shots=1)
  results=list(res)[0][::-1].split(" ")
  RA=results[0]
  RB=results[1]
  return RA,RB




In [18]:
def create_singlet(qr,cr,name='singlet'):
    singlet = QuantumCircuit(qr, cr, name=name)
    # singlet.x(qr[0])
    # singlet.x(qr[1])
    singlet.h(qr[0])
    singlet.cx(qr[0],qr[1])
    singlet.barrier(label="singlet_end")
    return singlet

def reorder(num_qubits, qc, qrA, qrB, crA, crB):
  # Manually reorder qubits for drawing to interleave qrA_i and qrB_i
  # Recreate a circuit that interleaves the qubits
  ordered_qr = []

  # Interleave qubits from qrA and qrB
  for i in range(num_qubits):
      ordered_qr.append(qrA[i])
      ordered_qr.append(qrB[i])

  # Create a new QuantumCircuit with reordered qubits
  qc_reordered = QuantumCircuit(ordered_qr, crA, crB)

  # Transfer operations from the original circuit to the reordered one
  for instruction in qc.data:
      operation = instruction.operation
      qargs = instruction.qubits
      cargs = instruction.clbits

      # Map the qubits to the new ordering
      new_qargs = [ordered_qr[ordered_qr.index(q)] for q in qargs]
      qc_reordered.append(operation, new_qargs, cargs)

  # Draw the reordered circuit
  # qc_reordered.draw(output='mpl', fold=-1)
  # qc_reordered.draw(fold=-1)
  return qc_reordered

def create_singlets(num_qubits):
  # Define quantum registers and classical registers
  qrA = QuantumRegister(num_qubits, name='qrA')
  qrB = QuantumRegister(num_qubits, name='qrB')
  crA = ClassicalRegister(num_qubits, name='crA')
  crB = ClassicalRegister(num_qubits, name='crB')

  # Create a quantum circuit with qrA, qrB, crA, and crB
  qc = QuantumCircuit(qrA, qrB, crA, crB)

  # Apply X gates to all qubits in qrA

  qc.x(qrA)
  qc.x(qrB)




  # Apply H gates to all qubits in qrA
  qc.h(qrA)
  # Apply CX (CNOT) gates between corresponding qubits in qrA and qrB
  qc.cx(qrA, qrB)
  # qc.z(0)
  # Add a barrier for clarity (optional)
  qc.barrier(label="singlets_end")
  return qc, qrA, qrB, crA, crB


In [19]:
# abPatterns = [
#     re.compile('..00$'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
#     re.compile('..01$'), # search for the '..01' output
#     re.compile('..10$'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
#     re.compile('..11$')  # search for the '..11' output
# ]
test=[(0,0),(0,1),(1,0),(1,1)]
# function that calculates CHSH correlation value
def chsh_corr(RA,RB,BA,BB):
    aliceMeasurementChoices=BA
    bobMeasurementChoices=BB

    # lists with the counts of measurement results
    # each element represents the number of (-1,-1), (-1,1), (1,-1) and (1,1) results respectively
    countA1B1 = [0, 0, 0, 0] # XW observable
    countA1B3 = [0, 0, 0, 0] # XV observable
    countA3B1 = [0, 0, 0, 0] # ZW observable
    countA3B3 = [0, 0, 0, 0] # ZV observable

    for i in range(len(RA)):
        # print(aliceMeasurementChoices[i],bobMeasurementChoices[i])
        valpair=(int(RB[i]),int(RA[i]))

        # if the spins of the qubits of the i-th singlet were projected onto the a_1/b_1 directions
        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if test[j]==valpair:
                    countA1B1[j] += 1

        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if test[j]==valpair:
                    countA1B3[j] += 1

        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if test[j]==valpair:
                    countA3B1[j] += 1

        # if the spins of the qubits of the i-th singlet were projected onto the a_3/b_3 directions
        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if test[j]==valpair:
                    countA3B3[j] += 1

    # number of the results obtained from the measurements in a particular basis
    total11 = sum(countA1B1)
    total13 = sum(countA1B3)
    total31 = sum(countA3B1)
    total33 = sum(countA3B3)

    # expectation values of XW, XV, ZW and ZV observables (2)
    expect11 = (countA1B1[0] - countA1B1[1] - countA1B1[2] + countA1B1[3])/total11 # -1/sqrt(2)
    expect13 = (countA1B3[0] - countA1B3[1] - countA1B3[2] + countA1B3[3])/total13 # 1/sqrt(2)
    expect31 = (countA3B1[0] - countA3B1[1] - countA3B1[2] + countA3B1[3])/total31 # -1/sqrt(2)
    expect33 = (countA3B3[0] - countA3B3[1] - countA3B3[2] + countA3B3[3])/total33 # -1/sqrt(2)

    # print(expect11, expect13, expect31, expect33)
    corr = expect11 - expect13 + expect31 + expect33 # calculate the CHSC correlation value (3)

    return corr

In [20]:
# # BA,BB range = {1,2,3}
# # RA,RB range ={0,1}


# def cal_CHSH_noabs(numberOfSinglets,BA,BB,RA,RB):
#   # Lists with the counts of measurement results
#   # Each element represents the number of (-1,-1), (-1,1), (1,-1), and (1,1) results respectively
#   A3B1 = [0, 0, 0, 0]  # XW observable
#   A3B4 = [0, 0, 0, 0]  # XV observable
#   A2B1 = [0, 0, 0, 0]  # ZW observable
#   A2B4 = [0, 0, 0, 0]  # ZV observable


#   def cal_prob(i, RA, RB):
#     P = 0
#     val_pair=(int(RA[i]), int(RB[i]))
#     if val_pair == (0, 0):
#       P += 1
#     elif val_pair == (0, 1):
#       P += 2
#     elif val_pair == (1, 0):
#       P += 3
#     elif val_pair == (1, 1):
#       P += 4
#     return P

#   count = []
#   for i in range(numberOfSinglets):
#       # If the spins of the qubits of the i-th singlet were projected onto the a[1]/b[1] directions
#       if (BA[i] == 1 and BB[i] == 2):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A3B1[j] += 1

#       if (BA[i] == 1 and BB[i] == 3):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A3B4[j] += 1

#       if (BA[i] == 0 and BB[i] == 2):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A2B1[j] += 1

#       # If the spins of the qubits of the i-th singlet were projected onto the a[3]/b[3] directions
#       if (BA[i] == 0 and BB[i] == 3):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A2B4[j] += 1

#   # Number of the results obtained from the measurements in a particular basis
#   total11 = sum([A3B1[0], A3B1[1], A3B1[2], A3B1[3]])
#   total13 = sum([A3B4[0], A3B4[1], A3B4[2], A3B4[3]])
#   total31 = sum([A2B1[0], A2B1[1], A2B1[2], A2B1[3]])
#   total33 = sum([A2B4[0], A2B4[1], A2B4[2], A2B4[3]])

#   # Expectation values of XW, XV, ZW, and ZV observables
#   expect31 = (A3B1[0] - A3B1[1] - A3B1[2] + A3B1[3]) / total11  # -1/sqrt(2)
#   expect34 = (A3B4[0] - A3B4[1] - A3B4[2] + A3B4[3]) / total13  # 1/sqrt(2)
#   expect21 = (A2B1[0] - A2B1[1] - A2B1[2] + A2B1[3]) / total31  # -1/sqrt(2)
#   expect24 = (A2B4[0] - A2B4[1] - A2B4[2] + A2B4[3]) / total33  # -1/sqrt(2)

#   # Calculate the CHSH correlation value
#   # corr = expect31 - expect34 + expect21 + expect24
#   corr = expect31 - expect34 + expect21 + expect24

#   # print(A3B1[0], A3B1[1], A3B1[2], A3B1[3],total11)
#   # print(A3B4[0], A3B4[1], A3B4[2], A3B4[3],total13)
#   # print(A2B1[0], A2B1[1], A2B1[2], A2B1[3],total31)
#   # print(A2B4[0], A2B4[1], A2B4[2], A2B4[3],total33)

#   # print(expect21, expect34, expect21, expect24, total11+total13+total31+total33)

#   # print(corr)
#   return corr

In [21]:
# def cal_CHSH_abs(numberOfSinglets, BA,BB,RA,RB):
#   # Lists with the counts of measurement results
#   # Each element represents the number of (-1,-1), (-1,1), (1,-1), and (1,1) results respectively
#   A3B1 = [0, 0, 0, 0]  # XW observable
#   A3B4 = [0, 0, 0, 0]  # XV observable
#   A2B1 = [0, 0, 0, 0]  # ZW observable
#   A2B4 = [0, 0, 0, 0]  # ZV observable


#   def cal_prob(i, RA, RB):
#     P = 0
#     val_pair=(int(RA[i]), int(RB[i]))
#     if val_pair == (0, 0):
#       P += 1
#     elif val_pair == (0, 1):
#       P += 2
#     elif val_pair == (1, 0):
#       P += 3
#     elif val_pair == (1, 1):
#       P += 4
#     return P

#   count = []
#   for i in range(numberOfSinglets):
#       # If the spins of the qubits of the i-th singlet were projected onto the a[1]/b[1] directions
#       if (BA[i] == 1 and BB[i] == 2):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A3B1[j] += 1

#       if (BA[i] == 1 and BB[i] == 3):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A3B4[j] += 1

#       if (BA[i] == 0 and BB[i] == 2):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A2B1[j] += 1

#       # If the spins of the qubits of the i-th singlet were projected onto the a[3]/b[3] directions
#       if (BA[i] == 0 and BB[i] == 3):
#           for j in range(4):
#               if cal_prob(i, RA, RB) == j + 1:
#                   A2B4[j] += 1

#   # Number of the results obtained from the measurements in a particular basis
#   total11 = sum([A3B1[0], A3B1[1], A3B1[2], A3B1[3]])
#   total13 = sum([A3B4[0], A3B4[1], A3B4[2], A3B4[3]])
#   total31 = sum([A2B1[0], A2B1[1], A2B1[2], A2B1[3]])
#   total33 = sum([A2B4[0], A2B4[1], A2B4[2], A2B4[3]])

#   # Expectation values of XW, XV, ZW, and ZV observables
#   expect31 = (A3B1[0] - A3B1[1] - A3B1[2] + A3B1[3]) / total11  # -1/sqrt(2)
#   expect34 = (A3B4[0] - A3B4[1] - A3B4[2] + A3B4[3]) / total13  # 1/sqrt(2)
#   expect21 = (A2B1[0] - A2B1[1] - A2B1[2] + A2B1[3]) / total31  # -1/sqrt(2)
#   expect24 = (A2B4[0] - A2B4[1] - A2B4[2] + A2B4[3]) / total33  # -1/sqrt(2)

#   # Calculate the CHSH correlation value
#   # corr = expect31 - expect34 + expect21 + expect24
#   corr = abs(expect31) + abs(expect34) + abs(expect21) + abs(expect24)

#   # print(A3B1[0], A3B1[1], A3B1[2], A3B1[3],total11)
#   # print(A3B4[0], A3B4[1], A3B4[2], A3B4[3],total13)
#   # print(A2B1[0], A2B1[1], A2B1[2], A2B1[3],total31)
#   # print(A2B4[0], A2B4[1], A2B4[2], A2B4[3],total33)

#   # print(expect21, expect34, expect21, expect24, total11+total13+total31+total33)

#   # print(corr)
#   return corr


In [22]:
# import numpy as np

# # Calculate the correlation between Alice and Bob's results for given basis
# def correlation(BA, RA, BB, RB, a_basis, b_basis):
#     # BA: Alice's measurement basis choices (0, 1, 2)
#     # RA: Alice's measurement results (1 for +1, 0 for -1)
#     # BB: Bob's measurement basis choices (1, 2, 3)
#     # RB: Bob's measurement results (1 for +1, 0 for -1)
#     # a_basis: specific basis of Alice to consider
#     # b_basis: specific basis of Bob to consider

#     # Initializing counts for correlation calculation
#     N_pp = N_mm = N_pm = N_mp = 0  # ++ (1,1), -- (0,0), +- (1,0), -+ (0,1)

#     # Loop through each measurement result and filter by selected bases
#     for a, rA, b, rB in zip(BA, RA, BB, RB):
#         if a == a_basis and b == b_basis:  # Filter by the selected basis pair
#             if rA == 1 and rB == 1:
#                 N_pp += 1
#             elif rA == 0 and rB == 0:
#                 N_mm += 1
#             elif rA == 1 and rB == 0:
#                 N_pm += 1
#             elif rA == 0 and rB == 1:
#                 N_mp += 1

#     # Total measurements for this pair of bases
#     N_total = N_pp + N_mm + N_pm + N_mp

#     # If there are no measurements, return 0 correlation
#     if N_total == 0:
#         return 0

#     # Calculate the correlation value E(a, b)
#     E_ab = (N_pp + N_mm - N_pm - N_mp) / N_total
#     return E_ab

# # Function to calculate the CHSH value based on specific basis pairings
# def calculate_CHSH_v2(BA, RA, BB, RB):
#     # BA: Alice's basis choices (0, 1, 2 corresponding to a_0, a_1, a_2)
#     # RA: Alice's measurement results (1 for +1, 0 for -1)
#     # BB: Bob's basis choices (1, 2, 3 corresponding to b_1, b_2, b_3)
#     # RB: Bob's measurement results (1 for +1, 0 for -1)

#     # Calculate the correlations for the chosen basis pairings
#     E_a0_b1 = correlation(BA, RA, BB, RB, 0, 1)  # E(a_0, b_1)
#     E_a0_b2 = correlation(BA, RA, BB, RB, 0, 2)  # E(a_0, b_2)
#     E_a1_b1 = correlation(BA, RA, BB, RB, 1, 1)  # E(a_1, b_1)
#     E_a1_b2 = correlation(BA, RA, BB, RB, 1, 2)  # E(a_1, b_2)

#     # Calculate the CHSH value (S) based on the correlations
#     S = E_a0_b1 - E_a0_b2 + E_a1_b1 + E_a1_b2

#     return S

# **Visualization**

In [23]:
import matplotlib.pyplot as plt

def plot_distribution(data_dict):
    """
    Plots the distribution of a dictionary where keys represent values and
    values represent counts.

    Args:
    data_dict (dict): A dictionary with the format {value: count}
    """

    # Extract the keys (values) and corresponding counts
    values = data_dict.keys()
    counts = list(data_dict.values())

    # Create a bar plot
    plt.figure(figsize=(10,6))
    plt.bar(values, counts, color='skyblue')

    # Labeling the plot
    plt.xlabel('Values', fontsize=14)
    plt.ylabel('Counts', fontsize=14)
    plt.title('Distribution of Values', fontsize=16)

    # Show the plot
    plt.show()

In [24]:
import matplotlib.pyplot as plt
import numpy as np

def plot_two_distributions(data_dict1, data_dict2, label1='Distribution 1', label2='Distribution 2'):
    """
    Plots the distribution of two dictionaries where keys represent values and
    values represent counts. Overlaps between the distributions are highlighted
    with a different color.

    Args:
    data_dict1 (dict): First dictionary with the format {value: count}
    data_dict2 (dict): Second dictionary with the format {value: count}
    label1 (str): Label for the first distribution (used in the legend).
    label2 (str): Label for the second distribution (used in the legend).
    """

    # Get the union of keys from both dictionaries
    all_values = sorted(set(data_dict1.keys()).union(set(data_dict2.keys())))

    # Get the counts from each dictionary, filling in 0 for missing values
    counts1 = [data_dict1.get(value, 0) for value in all_values]
    counts2 = [data_dict2.get(value, 0) for value in all_values]

    # Set the bar width for side-by-side plotting
    bar_width = 0.35

    # Positions for the bars
    indices1 = np.arange(len(all_values))
    indices2 = [i + bar_width for i in indices1]

    # Highlight overlapping values by determining if both counts are non-zero
    overlap_indices = [i for i, (c1, c2) in enumerate(zip(counts1, counts2)) if c1 > 0 and c2 > 0]

    # Create the bar plot for both distributions
    plt.figure(figsize=(20,6))

    # Plot distribution 1
    plt.bar(indices1, counts1, color='skyblue', width=bar_width, label=label1, alpha=0.7)

    # Plot distribution 2
    plt.bar(indices2, counts2, color='pink', width=bar_width, label=label2, alpha=0.7)

    # Highlight the overlap
    # for i in overlap_indices:
    #     plt.bar(indices1[i], counts1[i], color='blue', width=bar_width, alpha=0.9)
    #     plt.bar(indices2[i], counts2[i], color='red', width=bar_width, alpha=0.9)

    # Setting labels, title, and legend
    plt.xlabel('Values', fontsize=14)
    plt.ylabel('Counts', fontsize=14)
    plt.title('Comparison of Two Distributions with Highlighted Overlap', fontsize=16)

    # Align x-ticks to the center of the bars
    plt.xticks([i + bar_width/2 for i in indices1], all_values)

    # Add a legend
    plt.legend()

    # Show the plot
    plt.show()

# # Example usage
# data1 = {1: 10, 2: 20, 3: 15}
# data2 = {2: 18, 3: 14, 4: 6, 5: 10}
# plot_two_distributions(data1, data2, label1='Group A', label2='Group B')

# **Exp Functions**

In [25]:
def average_dict_counts(*dicts):
    from collections import defaultdict

    # Dictionary to store total counts and number of occurrences
    combined = defaultdict(lambda: [0, 0])  # [sum, count]

    # Iterate over each dictionary
    for d in dicts:
        for val, count in d.items():
            combined[val][0] += count  # Sum of counts
            combined[val][1] += 1      # Increment occurrence count

    # Calculate average and store in a new dictionary
    averaged_dict = {val: total / num for val, (total, num) in combined.items()}

    return averaged_dict


In [26]:
def create_dist_ent(res,BA,BB,chsh_acc=1,prob_acc=3):
  CHSH_dist={}
  re_dist={}
  for k,v in res.items():
    results=k[::-1].split(" ")
    RAi=results[0]
    RBi=results[1]
    # print(RAi,RBi)
    CHSH_i=round(chsh_corr(RAi,RBi,BA,BB),chsh_acc)
    re_i=relative_error_cal(CHSH_i,-2*math.sqrt(2))
    if(CHSH_i not in CHSH_dist):
      CHSH_dist[CHSH_i]=v
      re_dist[re_i]=v
    else:
      CHSH_dist[CHSH_i]=round(CHSH_dist[CHSH_i]+v,prob_acc)
      re_dist[re_i]=round(re_dist[re_i]+v,prob_acc)
  return CHSH_dist,re_dist
  # print(CHSH_i,v)

def create_dist_pm(res,BA,BB,RA,chsh_acc=1,prob_acc=3):
  CHSH2_dist={}
  re2_dist={}
  for k,v in res.items():
    CHSH_i=round(chsh_corr(RA,k[::-1],BA,BB),chsh_acc)
    re_i=relative_error_cal(CHSH_i,-2*math.sqrt(2))
    if(CHSH_i not in CHSH2_dist):
      CHSH2_dist[CHSH_i]=v
      re2_dist[re_i]=v
    else:
      CHSH2_dist[CHSH_i]=round(CHSH2_dist[CHSH_i]+v,prob_acc)
      re2_dist[re_i]=round(re2_dist[re_i]+v,prob_acc)
  return CHSH2_dist,re2_dist
  # print(CHSH_i,v)


In [27]:


def run_attack_mr_exp(num_qubits,BA,BB,BE,shots=1):
  qc, qrA, qrB, crA, crB=create_singlets(num_qubits)
  qc=apply_basis(num_qubits,qc,qrB,BA,BE)
  qc=apply_measurement(num_qubits,qc,qrA,qrB,crA,crB)
  RA,RE=get_result(qc)

  qc2, qrA2, qrB2, crA2, crB2=create_singlets(num_qubits)
  qc2=apply_basis(num_qubits,qc2,qrA,qrB,BE,BB)
  qc2=apply_measurement(num_qubits,qc2,qrA2,qrB2,crA2,crB2)

  user="Bob"
  for i in range(num_qubits):
    quantumChannel=entanglment_basis_mapB[BB[i]](qc2,qrB2,i)


  for i in range(num_qubits):
      quantumChannel=measure_cir(quantumChannel,qrB2,crB2,i)


  res2=sim_circuits(quantumChannel,shots=shots)
  return RA,res2

In [28]:
def apply_basis_Eve(num_qubits, qc,qrA,qrB,user_basis):
  for i in range(num_qubits):
    qc=entanglment_basis_mapA[user_basis[i]](qc,qrA,i)

  for i in range(num_qubits):
    qc=entanglment_basis_mapA[user_basis[i]](qc,qrB,i)

  qc.barrier(label="Basis")
  return qc


In [29]:
def run_exp(num_qubits,BA,BB,shots=1,noise=False,L=None):
  qc, qrA, qrB, crA, crB=create_singlets(num_qubits)
  qc=apply_basis(num_qubits,qc,qrA,qrB,BA,BB)
  qc=apply_measurement(num_qubits,qc,qrA,qrB,crA,crB)



  res=sim_circuits(qc,shots=shots,noise=noise,L=L)
  return qc, res
  # print(res)

  # RA,RB=get_result(qc)
  # CHSH=chsh_corr(RA,RB,BA,BB)
  # error=relative_error_cal(CHSH, -2*math.sqrt(2))
  # # errortotal+=error
  # print("normal",CHSH,error)


def eve_encode(num_qubits,BA,RE):
  alice_raw_key=RE
  alice_basis=BA

  user="Alice"
  # Creating registers with n qubits
  qr2 = QuantumRegister(num_qubits, name='qr2')
  crB2 = ClassicalRegister(num_qubits, name='crB2')
  quantumChannel = QuantumCircuit(qr2, crB2)

  for i in range(num_qubits):
    if alice_raw_key[i] == 1 or alice_raw_key[i] == '1':
        quantumChannel.x(qr2[i])

  for i in range(num_qubits):
    quantumChannel=entanglment_basis_mapA[alice_basis[i]](quantumChannel,qr2,i)

  quantumChannel.barrier(qr2,label="AliceSending")
  return qr2,crB2,quantumChannel


def run_attackexp(num_qubits,BA,BB,shots=1,noise=False,L=None):
  qc, qrA, qrB, crA, crB=create_singlets(num_qubits)
  qc=apply_basis_Eve(num_qubits,qc,qrA,qrB,BA)
  qc=apply_measurement(num_qubits,qc,qrA,qrB,crA,crB)
  RA,RE=get_result(qc)

  qr2,crB2,quantumChannel=eve_encode(num_qubits,BA,RE)

  user="Bob"
  for i in range(num_qubits):
    quantumChannel=entanglment_basis_mapB[BB[i]](quantumChannel,qr2,i)


  for i in range(num_qubits):
      quantumChannel=measure_cir(quantumChannel,qr2,crB2,i)


  res2=sim_circuits(quantumChannel,shots=shots,noise=noise,L=L)
  return RA,res2

In [30]:
def average_and_normalize_probabilities(*dicts):
    from collections import defaultdict

    # Dictionary to store the sum of probabilities and their occurrences
    combined = defaultdict(lambda: [0, 0])  # [sum of probabilities, count]

    # Iterate over each dictionary
    for d in dicts:
        for val, prob in d.items():
            combined[val][0] += prob  # Sum of probabilities
            combined[val][1] += 1     # Count occurrences

    # Average the probabilities for each value
    averaged_dict = {val: total / num for val, (total, num) in combined.items()}

    # Normalize the averaged probabilities to ensure their sum is 1
    total_sum = sum(averaged_dict.values())
    normalized_dict = {val: prob / total_sum for val, prob in averaged_dict.items()}
    normalized_dict = {val: round(prob,3)  for val, prob in normalized_dict.items()}

    return normalized_dict

In [31]:
def bitwise_probability_distribution(res):
    from collections import defaultdict

    # Initialize a dictionary where each position will hold counts for '0' and '1'
    bitwise_distribution = defaultdict(lambda: {'0': 0, '1': 0})

    # Iterate over each bit string and its associated probability
    for bit_string, probability in res.items():
        for i, bit in enumerate(bit_string):  # Loop through each bit in the string
            bitwise_distribution[i][bit] = round(bitwise_distribution[i][bit]+probability,3)  # Add the probability to the corresponding bit ('0' or '1')

    # Convert defaultdict to normal dict before returning
    return dict(bitwise_distribution)


# **Encoding_Ent_Attack**

In [32]:
# def apply_basis_Eve(num_qubits, qc,qrA,qrB,user_basis):
#   for i in range(num_qubits):
#     qc=entanglment_basis_mapA[user_basis[i]](qc,qrA,i)

#   for i in range(num_qubits):
#     qc=entanglment_basis_mapA[user_basis[i]](qc,qrB,i)

#   qc.barrier(label="Basis")
#   return qc


In [33]:
def run_attackexp_2(num_qubits,BA,BB,shots=1,noise=False,L=None):
  qc, qrA, qrB, crA, crB=create_singlets(num_qubits)
  qc=apply_basis_Eve(num_qubits,qc,qrA,qrB,BA)
  qc=apply_measurement(num_qubits,qc,qrA,qrB,crA,crB)
  RA,RE=get_result(qc)

  qr2,crB2,quantumChannel=eve_encode(num_qubits,BA,RE)

  user="Bob"
  for i in range(num_qubits):
    quantumChannel=entanglment_basis_mapB[BB[i]](quantumChannel,qr2,i)


  for i in range(num_qubits):
      quantumChannel=measure_cir(quantumChannel,qr2,crB2,i)


  res2=sim_circuits(quantumChannel,shots=shots,noise=noise,L=L)
  return RA,res2,quantumChannel

In [45]:
shots=100
num_qubits=1000

dist_NoEve_all=[]
dist_Eve_all=[]

redist_NoEve_all=[]
redist_Eve_all=[]

for i in range(1):
  BA = aliceMeasurementChoices(num_qubits)
  BB   = bobMeasurementChoices(num_qubits)

  qc, res=run_exp(num_qubits,BA,BB,shots=shots,noise=False,L=None)
  RA,res2,quantumChannel=run_attackexp_2(num_qubits,BA,BB,shots=shots,noise=False,L=None)

  CHSHdist_NoEve,redist_NoEve=create_dist_ent(res,BA,BB)
  CHSHdist_Eve,redist_Eve=create_dist_pm(res2,BA,BB,RA)

  dist_NoEve_all.append(CHSHdist_NoEve)
  dist_Eve_all.append(CHSHdist_Eve)

  redist_NoEve_all.append(redist_NoEve)
  redist_Eve_all.append(redist_Eve)

In [46]:
dist_NoEve_final=average_and_normalize_probabilities(*dist_NoEve_all)
dist_Eve_final=average_and_normalize_probabilities(*dist_Eve_all)

redist_NoEve_final=average_and_normalize_probabilities(*redist_NoEve_all)
redist_Eve_final=average_and_normalize_probabilities(*redist_Eve_all)

In [47]:
print(dist_NoEve_final)
print(dist_Eve_final)

{-2.8: 0.31, -2.7: 0.13, -2.9: 0.2, -3.0: 0.2, -2.5: 0.03, -3.1: 0.05, -2.6: 0.07, -3.2: 0.01}
{-2.7: 0.15, -2.9: 0.31, -2.5: 0.06, -3.0: 0.16, -2.8: 0.23, -2.6: 0.06, -3.1: 0.03}


In [55]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

def plot_qber_distributions_with_patterns(data_dict1, data_dict2, label1='Distribution 1', label2='Distribution 2', output_file='output.pdf'):
    """
    Plots two distributions of QBER values with probability as percentage,
    highlighting overlaps with specific hatch patterns for easy distinction,
    and saves the figure to a PDF.

    Args:
    data_dict1 (dict): First dictionary with the format {QBER: probability}
    data_dict2 (dict): Second dictionary with the format {QBER: probability}
    label1 (str): Label for the first distribution (used in the legend).
    label2 (str): Label for the second distribution (used in the legend).
    output_file (str): File name to save the plot as a PDF.
    """

    all_values = sorted(set(data_dict1.keys()).union(set(data_dict2.keys())))

    probs1 = [data_dict1.get(value, 0) * 100 for value in all_values]
    probs2 = [data_dict2.get(value, 0) * 100 for value in all_values]

    # Set bar width and x-axis positions
    bar_width = 0.4
    indices = np.arange(len(all_values))

    # Create the bar plot
    plt.figure(figsize=(12, 6))


    plt.bar(indices - bar_width / 2, probs1, bar_width, color='white', edgecolor='blue', hatch='//', label=label1, alpha=0.7)
    plt.bar(indices + bar_width / 2, probs2, bar_width, color='orange', edgecolor='black', hatch='.', label=label2, alpha=0.7)

    # Set labels, title, and legend
    plt.xlabel('CHSH value', fontsize=14)
    plt.ylabel('Probability (%)', fontsize=14)
    plt.title('Distributions for CHSH value for Normal vs Attack', fontsize=16)
    plt.xticks(indices, [f"{value:.3f}" for value in all_values], fontsize=12)  # Format QBER values with three decimals
    plt.yticks(fontsize=12)

    # Custom legend with specific patterns
    legend_patches = [
        mpatches.Patch(facecolor='white', edgecolor='blue', hatch='//', label=label1),
        mpatches.Patch(facecolor='orange', edgecolor='black', hatch='.', label=label2),
    ]
    plt.legend(handles=legend_patches, fontsize=12)

    # Save the plot to a PDF
    plt.tight_layout()
    plt.savefig(output_file, format='pdf')
    plt.close()

In [56]:
# Plotting the distributions
plot_qber_distributions_with_patterns(dist_Eve_final, dist_NoEve_final, label1='Distribution for Encoding over Entanglement Attack', label2='Distribution for typical working of protocols without Eve')

# **Failed_Simualtion_for Misc trace**

In [ ]:
def SendQ_eve(num_qubits,eve_raw_key,eve_basis,user="Eve"):
  # Creating registers with n qubits
  qr = QuantumRegister(num_qubits, name='qr')
  crB = ClassicalRegister(num_qubits, name='crB')
  quantumChannel = QuantumCircuit(qr, crB)

  for i in range(num_qubits):
    if eve_raw_key[i] == '1':
        quantumChannel.x(qr[i])

    if eve_basis[i] == '1':
        quantumChannel.h(qr[i])
    elif(eve_basis[i] == 'x'):
      quantumChannel.h(qr[i])
      quantumChannel.t(qr[i])
      quantumChannel.h(qr[i])
      quantumChannel.s(qr[i])

  quantumChannel.barrier(qr,label="EveSending")
  return qr,crB,quantumChannel

In [ ]:
karaw

In [ ]:
num_qubits = 1000
for _ in range(100):
  karaw, SA_0,SB_0,BA_0,BB_0=QKD_Session(num_qubits,shots=1)

  eve_raw_key=[0] * num_qubits
  BA=bitarray2str(BA_0)
  BE=""

  for i,ind in enumerate(SA_0[1]):
    eve_raw_key[ind]=SA_0[0][i]


  for i in range(num_qubits):
    if(i in SA_0[1]):
      BE+='x'
    else:
      BE+=BA[i]


  bob_basis   = "".join(str(i) for i in list(np.random.randint(2, size=num_qubits)))
  #Quantum Channel
  qr,crB,quantumChannelA2B=SendQ_eve(num_qubits,eve_raw_key,BE,user="Eve")
  quantumChannelB=ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannelA2B,user="Bob")
  resultBshots=sim_circuits(quantumChannelB,shots=1)

  resultB=list(resultBshots.keys())[0][::-1]


  # for i,v in resultBshots.items():
  #   resultB=i[::-1]
  KsiftedB=Sift(num_qubits,resultB,bitarray2str(BA_0),bob_basis)
  KB,SBv,SBi=Sample(KsiftedB,bitarray2str(BA_0),bob_basis)
  # print(len(SA_0[0]),len(SBv))
  if(len(SA_0[0])==len(SBv)):

    qber=calculate_qber(SA_0[0],SBv)
    print(qber)
    # else:
    #   print("Error")

0.42857142857142855
0.6030534351145038


In [ ]:
# KsiftedB=Sift(num_qubits,resultB,bitarray2str(BA_0),bob_basis)
# KB,SBv,SBi=Sample(KsiftedB,bitarray2str(BA_0),bob_basis)

# if(len(SA_0[0])==len(SBv)):
#   qber=calculate_qber(SA_0[0],SBv)
#   print(qber)
# else:
#   print("Error")

In [ ]:
num_qubits = 100
bob_basis   = list(np.random.randint(2, size=num_qubits))

In [ ]:
for i in range(1000):
  alice_raw_key   = list(np.random.randint(2, size=num_qubits))
  alice_basis = list(np.random.randint(2, size=num_qubits))

  #Quantum Channel
  qr,crB,quantumChannelA2B=SendQ(num_qubits,alice_raw_key,alice_basis,user="Alice")
  quantumChannelB=ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannelA2B,user="Bob")
  resultB=list(sim_circuits(quantumChannelB,shots=1).keys())[0][::-1]

  #Sift
  KsiftedB=Sift(num_qubits,resultB,alice_basis,bob_basis)
  SB,KB=Sample(KsiftedB,alice_basis,bob_basis)

  KsiftedA=Sift(num_qubits,alice_raw_key,alice_basis,bob_basis)
  SA,KA=Sample(KsiftedA, alice_raw_key, bob_basis)
  # print(SB)
  # print(SA)
  if(len(SA)==len(SB)):
    print(calculate_qber(SA,SB))

NameError: name 'num_qubits' is not defined

In [ ]:
quantumChannelB=ReceiveQ(num_qubits,bob_basis, qr,crB,quantumChannelA2B,user="Bob")
KsiftedB=Sift(num_qubits,resultB,BA_0,bob_basis)

In [ ]:
#Sift

# KsiftedA=Sift(num_qubits,"".join(str(i) for i in alice_raw_key),BA_0,bob_basis)

#ParameterEstimation
##Sample

# SA,KA=Sample(KsiftedA,BA_0,bob_basis)
SA=SA_0
SB,KB=Sample(KsiftedB,BA_0,bob_basis)

# print(SA)
# print(SB)

print(len(SA),len(SB))

# print(calculate_qber(SA,SB))
# print(KsiftedA, KsiftedB)
# CheckEveB=hamming_distance(SA,SB)/len(SB)
# print(CheckEveB)
# print(resultB)

In [ ]:
import random


# Basis choices for measurement (0: Z-basis, 1: X-basis)
def random_bases(n):
    return [random.choice([0, 1]) for _ in range(n)]

def BBM92(n_qubits,alice_bases,bob_bases):
  # Create the entangled state
  def create_entangled_pair():
      qc = QuantumCircuit(2, 2)
      qc.h(0)  # Apply Hadamard gate on qubit 0
      qc.cx(0, 1)  # CNOT gate to entangle qubits
      return qc

  # Measure based on chosen basis
  def measure_in_basis(qc, basis, qubit, clbit):
      if basis == 1:
          qc.h(qubit)  # Apply Hadamard for X-basis measurement
      qc.measure(qubit, clbit)  # Measure in the chosen basis

  # Initialize key lists and bases
  alice_key = []
  bob_key = []


  # Run the protocol for each qubit pair
  for i in range(n_qubits):
      qc = create_entangled_pair()

      # Alice and Bob measure in their chosen basis
      measure_in_basis(qc, alice_bases[i], 0, 0)
      measure_in_basis(qc, bob_bases[i], 1, 1)

      # Run the quantum circuit
      backend = AerSimulator(method='matrix_product_state')
      job = backend.run(qc, shots=1)
      results = job.result()

      counts = results.get_counts()
      outcome = list(counts.keys())[0]

      # Record the bits if bases match
      if alice_bases[i] == bob_bases[i]:
          alice_key.append(int(outcome[1]))  # Alice's result (qubit 0)
          bob_key.append(int(outcome[0]))    # Bob's result (qubit 1)


  return alice_key,bob_key

In [ ]:
# Number of qubit pairs to be used for key generation
n_qubits = 100
alice_bases = random_bases(n_qubits)
bob_bases = random_bases(n_qubits)

alice_key,bob_key=BBM92(n_qubits,alice_bases,bob_bases)


# Display the final key
print("Alice's key:", alice_key)
print("Bob's key:  ", bob_key)

# Compare keys to confirm they match (in a real implementation, privacy amplification would follow)


Alice's key: [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Bob's key:   [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]


In [ ]:

# Number of qubit pairs to be used for key generation
n_qubits = 100

# Define basis sets for Alice and Bob based on Bell's theorem settings
# Basis angles for measurements:
# Alice: A1 = 0°, A2 = 45°
# Bob: B1 = 22.5°, B2 = 67.5°
alice_bases = [random.choice([0, 45]) for _ in range(n_qubits)]
bob_bases = [random.choice([22.5, 67.5]) for _ in range(n_qubits)]

# Create entangled pair
def create_entangled_pair():
    qc = QuantumCircuit(2, 2)
    qc.h(0)  # Hadamard gate on qubit 0
    qc.cx(0, 1)  # CNOT gate to create entanglement
    return qc

# Measure in basis
def measure_in_basis(qc, angle, qubit, clbit):
    # Rotate qubit by specified angle (converted from degrees to radians)
    qc.ry(2 * angle * (3.14159265 / 180), qubit)
    qc.measure(qubit, clbit)  # Measure in rotated basis

# Initialize key lists
alice_key = []
bob_key = []

# Run the protocol for each qubit pair
backend = AerSimulator(method='matrix_product_state')
for i in range(n_qubits):
    qc = create_entangled_pair()

    # Apply Alice and Bob's measurement angles
    measure_in_basis(qc, alice_bases[i], 0, 0)
    measure_in_basis(qc, bob_bases[i], 1, 1)

    # Execute the quantum circuit
    job = backend.run(qc, shots=1)
    results = job.result()
    counts = results.get_counts()
    outcome = list(counts.keys())[0]

    # Interpret results based on measurement outcomes
    alice_result = int(outcome[1])  # Alice's result (qubit 0)
    bob_result = int(outcome[0])    # Bob's result (qubit 1)

    # If the measurement bases (angles) are compatible, add to the key
    # Compatible pairs are (A1, B1) or (A2, B2)
    if (alice_bases[i] == 0 and bob_bases[i] == 22.5) or (alice_bases[i] == 45 and bob_bases[i] == 67.5):
        alice_key.append(alice_result)
        bob_key.append(bob_result)

# Display the final key
print("Alice's key:", alice_key)
print("Bob's key:  ", bob_key)

# In a real implementation, they would compare a subset of the key to check for eavesdropping,
# and then perform privacy amplification.


Alice's key: [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1]
Bob's key:   [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1]
